# 离中趋势测量
By Evgenia "Jenny" Nitishinskaya, Maxwell Margenot, and Delaney Mackenzie.

Part of the Quantopian Lecture Series:

* [www.quantopian.com/lectures](https://www.quantopian.com/lectures)
* [github.com/quantopian/research_public](https://github.com/quantopian/research_public)


---

离散度度量一组数据的分布情况。这在金融领域尤为重要，因为衡量风险的主要方式之一，就是看历史上回报率的分布情况。如果回报率围绕一个中心价值非常紧张，那么我们就没有那么多理由担心。如果回报到处都是，那是有风险的。

低离散度的数据主要集中在平均值附近，而高离散度的数据则表示有许多非常大和非常小的值。

让我们生成一个随机整数数组。

In [1]:
# Import libraries
import numpy as np

np.random.seed(121)

In [2]:
# Generate 20 random integers < 100
X = np.random.randint(100, size=20)

# Sort them
X = np.sort(X)
print ('X: %s' %(X))

mu = np.mean(X)
print('Mean of X:', mu) 

X: [ 3  8 34 39 46 52 52 52 54 57 60 65 66 75 83 85 88 94 95 96]
Mean of X: 60.2


# 极差

极差就是数据集中最大值和最小值之间的差值。毫不奇怪，它对异常值非常敏感。我们将使用`numpy`的peak to peak (ptp)函数。



In [3]:
print ('Range of X: %s' %(np.ptp(X)))

Range of X: 93


# 平均绝对偏差(MAD)

平均绝对偏差是观测距离算术平均值的平均值。我们用标准差的绝对值，也就是均值以上5个，均值以下5个，都贡献了5，否则标准差总和为0。

$$ MAD = \frac{\sum_{i=1}^n |X_i - \mu|}{n} $$

其中$n$是观测值的个数，$\mu$是它们的均值。

In [4]:
abs_dispersion = [np.abs(mu - x) for x in X]
MAD = np.sum(abs_dispersion)/len(abs_dispersion)
print( 'Mean absolute deviation of X:', MAD)

Mean absolute deviation of X: 20.520000000000003


# 方差和标准差

方差$\sigma^2$定义为均值周围平方偏差的平均值:
$$ \sigma^2 = \frac{\sum_{i=1}^n (X_i - \mu)^2}{n} $$

这有时比平均绝对偏差更方便，因为绝对值是不可微的，而平方是光滑的，而且一些优化算法依赖于可微性。

标准差被定义为方差的平方根，$\sigma$，它是两者中比较容易解释的，因为它与观测值的单位相同。

In [5]:
print ('Variance of X:', np.var(X))
print ('Standard deviation of X:', np.std(X))

Variance of X: 670.16
Standard deviation of X: 25.8874486962


一种解释标准差的方法是参照契比雪夫不等式。这告诉我们，对于任意$k> $1，观测值位于均值的$k$个标准差范围内的样本占比至少为$1 - 1/k^2$。

我们来验证一下，对于我们的数据集，这是正确的。

In [5]:
k = 1.25
dist = k*np.std(X)
l = [x for x in X if abs(x - mu) <= dist]
print ('Observations within', k, 'stds of mean:', l)
print ('Confirming that', float(len(l))/len(X), '>', 1 - 1/k**2)

Observations within 1.25 stds of mean: [34, 39, 46, 52, 52, 52, 54, 57, 60, 65, 66, 75, 83, 85, 88]
Confirming that 0.75 > 0.36


在这种情况下，契比雪夫不等式给出的界似乎相当宽松。这个界限很少是严格的，但它是有用的，因为它适用于所有的数据集和分布。

# 半方差和半标准差

虽然方差和标准差告诉我们一个量的波动性有多大，但它们不能区分向上的偏差和向下的偏差。通常情况下，比如资产回报率，我们更担心向下的偏差。这是通过半方差和半标准差来解决的，它们只计算低于平均值的观测值。半方差定义为
$$ \frac{\sum_{X_i < \mu} (X_i - \mu)^2}{n_<} $$
其中$n_<$是小于均值的观测值的个数。半标准差是半方差的平方根。

In [7]:
# Because there is no built-in semideviation, we'll compute it ourselves
lows = [e for e in X if e <= mu]

semivar = np.sum( (lows - mu) ** 2 ) / len(lows)

print('Semivariance of X:', semivar) 
print('Semideviation of X:', np.sqrt(semivar)) 

Semivariance of X: 689.5127272727273
Semideviation of X: 26.258574357202395


一个相关的概念是目标半方差(和目标半标准差)，其中我们平均到目标值以下的值的距离:
$$ \frac{\sum_{X_i < B} (X_i - B)^2}{n_{<B}} $$


In [8]:
B = 19
lows_B = [e for e in X if e <= B]
semivar_B = sum(map(lambda x: (x - B)**2,lows_B))/len(lows_B)

print ('Target semivariance of X:', semivar_B)
print ('Target semideviation of X:', np.sqrt(semivar_B))

Target semivariance of X: 188.5
Target semideviation of X: 13.729530217745982


# 这些只是估计

所有这些计算都会给出样本统计，也就是数据样本的标准差。这是否反映了当前真实的总体标准差并不总是很明显，需要更多的努力来确定。这在金融领域尤其成问题，因为所有数据都是时间序列，均值和方差可能随时间变化。这里有许多不同的技巧和微妙之处，其中一些在[quantlecture Series](https://www.quantopian.com/lectures)的其他讲座中有介绍。

一般来说，不要假设因为你的样本是正确的，它将继续保持正确。

## 参考
* "Quantitative Investment Analysis", by DeFusco, McLeavey, Pinto, and Runkle